In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import random

In [ ]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [ ]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash', how='inner')

In [ ]:
purchased_coupons_train.head(3)

In [ ]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [ ]:
purchased_coupons_train = purchased_coupons_train[features]

In [ ]:
df = purchased_coupons_train.copy()
df = df[:1000] 

In [ ]:
# df['DISCOUNT_PRICE'] = 1 / np.log10(df['DISCOUNT_PRICE'])
# df['PRICE_RATE'] = (df['PRICE_RATE'] / 100) ** 2
# features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

In [ ]:
# categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
#                 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
#                 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
# df_categoricals = df[categoricals]
# df_categoricals = pd.get_dummies(df_categoricals, dummy_na=False)

In [ ]:
# continuous = list(set(features) - set(categoricals))
# df = pd.concat([df[continuous], df_categoricals], axis=1)

In [ ]:
# NAN_SUBSTITUTION_VALUE = 1
# df = df.fillna(NAN_SUBSTITUTION_VALUE)

In [ ]:
df_temp = df.copy()
# df_temp = df_temp.drop(['COUPON_ID_hash', 'USER_ID_hash'], axis=1)

categorical_cols = ['COUPON_ID_hash', 'USER_ID_hash', 'GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name']

In [ ]:
df_temp

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: le.fit_transform(col))
df_temp = df_temp.fillna(0)
df_temp

In [ ]:
l = df_temp.shape[0]
f = len(df_temp.columns)
t = AnnoyIndex(f, 'angular')

In [ ]:
for i in range(l):
    v = [df_temp.values[i][z] for z in range(f)]
    t.add_item(i, v)

In [ ]:
t.build(5)

In [ ]:
#sim = t.get_nns_by_item(0, 5, include_distances=True)
sim = t.get_nns_by_item(0, 5)
sim

In [ ]:
df_temp.loc[sim]